In [37]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import geojson
import geopandas as gpd
import pandas as pd
import numpy as np
from time import time
import leafmap.foliumap as leafmap
import supervision as sv
from garuda.od import ConfusionMatrix
from IPython.display import Markdown

In [38]:
region = 'sylhet'
prediction_file = f"../data/predict/processed_labels/{region}/predictions_0.25.geojson"
print(f"Region: {region}")
print(f"Prediction file: {prediction_file}")

Region: sylhet
Prediction file: ../data/predict/processed_labels/sylhet/predictions_0.25.geojson


In [39]:
prediction_gdf = gpd.read_file(prediction_file) 
prediction_gdf['yolo_label'] = prediction_gdf['yolo_label'].apply(lambda x: np.array(list(map(float, x.split("_")))))
print(len(prediction_gdf))
print(prediction_gdf.crs)
prediction_gdf.head()



202
EPSG:3857


,x,y,class_name,confidence,yolo_label,geometry
0,10225442,2784751,Zigzag,0.782646,"[2.0, 0.354836, 0.999618, 0.368349, 0.99992, 0...","POLYGON ((10224998.447 2783223.853, 10225039.7..."
1,10222690,2793006,Zigzag,0.832081,"[2.0, 0.679098, 0.238839, 0.692999, 0.241333, ...","POLYGON ((10223238.139 2793805.119, 10223280.6..."
2,10225442,2801261,Zigzag,0.842850,"[2.0, 0.275691, 0.445727, 0.278613, 0.43005, 0...","POLYGON ((10224756.463 2801427.762, 10224765.3..."
3,10214435,2801261,FCBK,0.454749,"[1.0, 0.00115462, 0.294584, 0.0139894, 0.30257...","POLYGON ((10212910.141 2801889.879, 10212949.3..."
4,10228194,2801261,Zigzag,0.849970,"[2.0, 0.83096, 0.455518, 0.844085, 0.457745, 0...","POLYGON ((10229205.92 2801397.826, 10229246.05..."


In [40]:
prediction_gdf.crs.to_string()
print(prediction_gdf.tail(2))

            x        y class_name  confidence  \
200  10181414  2886565     Zigzag    0.785869   
201  10170407  2894820     Zigzag    0.903176   

                                            yolo_label  \
200  [2.0, 0.722989, 0.550203, 0.731152, 0.535728, ...   
201  [2.0, 0.855249, 0.529158, 0.869929, 0.542203, ...   

                                              geometry  
200  POLYGON ((10182096.339 2886412.052, 10182121.2...  
201  POLYGON ((10171493.79 2894731.596, 10171538.67...  


In [41]:
prediction_gdf.reset_index(inplace=True, drop=True)
print(prediction_gdf.head(2))

          x        y class_name  confidence  \
0  10225442  2784751     Zigzag    0.782646   
1  10222690  2793006     Zigzag    0.832081   

                                          yolo_label  \
0  [2.0, 0.354836, 0.999618, 0.368349, 0.99992, 0...   
1  [2.0, 0.679098, 0.238839, 0.692999, 0.241333, ...   

                                            geometry  
0  POLYGON ((10224998.447 2783223.853, 10225039.7...  
1  POLYGON ((10223238.139 2793805.119, 10223280.6...  


In [42]:
intersection_gdf = gpd.sjoin(prediction_gdf, prediction_gdf, predicate="intersects")[['index_right']].reset_index()
intersection_gdf = intersection_gdf[intersection_gdf['index'] < intersection_gdf['index_right']]
len(intersection_gdf)

4

In [43]:
def get_iou(row):
    left_polygon = prediction_gdf.iloc[int(row['index'])]['geometry']
    right_polygon = prediction_gdf.iloc[int(row['index_right'])]['geometry']
    left_area = left_polygon.area
    right_area = right_polygon.area
    iou = left_polygon.intersection(right_polygon).area / left_polygon.union(right_polygon).area
    return iou, left_area, right_area

intersection_gdf[['iou', 'left_area', 'right_area']] = intersection_gdf.apply(get_iou, axis=1, result_type="expand")
intersection_gdf = intersection_gdf[intersection_gdf['iou'] > 0.33]
len(intersection_gdf)
print(intersection_gdf.head(2))

    index  index_right       iou    left_area   right_area
82     81           83  0.372959  1545.053932  3105.626457
83     81           87  0.472139  1545.053932  2757.904602


In [44]:
drop_idx = intersection_gdf.apply(lambda x: x['index'] if x['left_area'] < x['right_area'] else x['index_right'], axis=1).astype(int).values
print(len(prediction_gdf))
prediction_gdf.drop(drop_idx, inplace=True)
print(len(prediction_gdf))
display(prediction_gdf.head(2))

202
200


,x,y,class_name,confidence,yolo_label,geometry
0,10225442,2784751,Zigzag,0.782646,"[2.0, 0.354836, 0.999618, 0.368349, 0.99992, 0...","POLYGON ((10224998.447 2783223.853, 10225039.7..."
1,10222690,2793006,Zigzag,0.832081,"[2.0, 0.679098, 0.238839, 0.692999, 0.241333, ...","POLYGON ((10223238.139 2793805.119, 10223280.6..."


In [45]:
final_kilns=prediction_gdf.to_crs(epsg=4326)
display(final_kilns.head(2))

,x,y,class_name,confidence,yolo_label,geometry
0,10225442,2784751,Zigzag,0.782646,"[2.0, 0.354836, 0.999618, 0.368349, 0.99992, 0...","POLYGON ((91.85272 24.24445, 91.85309 24.24444..."
1,10222690,2793006,Zigzag,0.832081,"[2.0, 0.679098, 0.238839, 0.692999, 0.241333, ...","POLYGON ((91.83691 24.33109, 91.83729 24.33103..."


In [46]:
import os
save_handvalidation_dir = f"../hand_validation/{region}"
os.makedirs(save_handvalidation_dir, exist_ok=True)

In [47]:
len(final_kilns)

200

In [48]:
version = "v1"  
batch_size = 500
for i in range(0, len(final_kilns), batch_size):
    end = min(i+batch_size, len(final_kilns))
    final_kilns.iloc[i:i+batch_size].to_file(f"../hand_validation/{region}/{version}_potential_kiln_{region}_{i}_{end-1}.geojson", driver='GeoJSON')

GARUDA INFO     : Created 200 records
